# Convert daily date range into dekads
DatetimeIndex --> datetime64 array 

In [1]:
import pandas as pd
import numpy as np

begin = pd.datetime(2002,7,1)
end = pd.datetime(2016,12,31)

dtrange = pd.date_range(begin, end)

p1 = np.random.rand(len(dtrange)) + 5
p2 = np.random.rand(len(dtrange)) + 10

df = pd.DataFrame({'p1': p1, 'p2': p2}, index=dtrange)

In [2]:
dtrange

DatetimeIndex(['2002-07-01', '2002-07-02', '2002-07-03', '2002-07-04',
               '2002-07-05', '2002-07-06', '2002-07-07', '2002-07-08',
               '2002-07-09', '2002-07-10',
               ...
               '2016-12-22', '2016-12-23', '2016-12-24', '2016-12-25',
               '2016-12-26', '2016-12-27', '2016-12-28', '2016-12-29',
               '2016-12-30', '2016-12-31'],
              dtype='datetime64[ns]', length=5298, freq='D')

In [3]:
type(p1)

numpy.ndarray

In [4]:
d = df.index.day - np.clip((df.index.day-1) // 10, 0, 2)*10 - 1
date = df.index.values - np.array(d, dtype="timedelta64[D]")
df.groupby(date).mean()

p1         p2
2002-07-01  5.447714  10.415188
2002-07-11  5.495729  10.544942
2002-07-21  5.446090  10.485094
2002-08-01  5.567267  10.345632
2002-08-11  5.383004  10.378629
...              ...        ...
2016-11-11  5.505803  10.555734
2016-11-21  5.471855  10.623065
2016-12-01  5.517205  10.557509
2016-12-11  5.611644  10.712101
2016-12-21  5.412262  10.467670

[522 rows x 2 columns]

In [5]:
date

array(['2002-07-01T00:00:00.000000000', '2002-07-01T00:00:00.000000000',
       '2002-07-01T00:00:00.000000000', ...,
       '2016-12-21T00:00:00.000000000', '2016-12-21T00:00:00.000000000',
       '2016-12-21T00:00:00.000000000'], dtype='datetime64[ns]')

In [ ]:
df.index.dayofyear

# Resample daily files in folders to dekads
dekad_resample function takes the raster mean of daily files in dekad chunks and rewrites dekad files to a new folder.
root_dir is a directory of folders of daily files separated by year/month and out_dir combines all outputs into a single folder. 

In [6]:
import rasterio
import numpy as np
from glob import glob
import os
from rasterio.plot import show

In [ ]:
def read_file(file):
    with rasterio.open(file) as src:
        return(src.read())

In [2]:
def dekad_resample(root_dir, out_dir, var):
    folds = [fold for fold in sorted(os.listdir(root_dir))]
    for fold in folds:
        os.chdir(root_dir + fold + '/')
        files = [file for file in sorted(os.listdir(os.getcwd()))]
        dekad1 = [read_file(x) for x in files[0:10]]
        dekad2 = [read_file(x) for x in files[10:20]]
        dekad3 = [read_file(x) for x in files[20:]]
        
     # Perform averaging
        dekad1_out = np.mean(dekad1, axis=0)
        dekad2_out = np.mean(dekad2, axis=0)
        dekad3_out = np.mean(dekad3, axis=0)

        with rasterio.open(files[0]) as src:
            meta = src.meta

        meta.update(dtype=rasterio.float32)

        # Write output file
        with rasterio.open(out_dir + var + fold + '_01.tif', 'w', **meta) as dst:
            dst.write(dekad1_out.astype(rasterio.float32))

        with rasterio.open(out_dir + var + fold + '_02.tif', 'w', **meta) as dst:
            dst.write(dekad2_out.astype(rasterio.float32))

        with rasterio.open(out_dir + var + fold + '_03.tif', 'w', **meta) as dst:
            dst.write(dekad3_out.astype(rasterio.float32))


In [3]:
tmax_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Months/' 
out_tmax_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Dekads/'


In [ ]:
sm_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Months/' 
out_sm_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Dekads/' 

In [4]:
dekad_resample(tmax_dir, out_tmax_dir, var = 'Tmax_')

/home/rgreen/miniconda3/envs/edm/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
